In [11]:
import pandas as pd

df = pd.read_csv('../Data/orders.csv', sep=';')
df.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,31/07/2012,31/07/2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,230965,7,0,7621845,93357,Critical
1,26341,IN-2013-77878,05/02/2013,07/02/2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709395,9,1,-288765,92363,Critical
2,25330,IN-2013-71249,17/10/2013,18/10/2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175171,9,1,919971,91549,Medium
3,13524,ES-2013-1579342,28/01/2013,30/01/2013,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",289251,5,1,-9654,91016,Medium
4,47221,SG-2013-4320,05/11/2013,06/11/2013,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",283296,8,0,31152,90304,Critical


## 1.Column Name Cleaning

In [25]:

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
df.columns


Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'city', 'state', 'country',
       'market', 'region', 'product_id', 'category', 'sub_category',
       'product_name', 'sales', 'quantity', 'discount', 'profit',
       'shipping_cost', 'order_priority'],
      dtype='object')

## 2.Remove Duplicate Rows


In [19]:
# Check for duplicate rows
duplicates = df.duplicated()
print("Total duplicate rows:", duplicates.sum())

# Drop duplicate rows if any
df = df.drop_duplicates()


Total duplicate rows: 0


## 3.Handle Missing Values (Nulls)

In [20]:
# Check for missing values in each column
df.isnull().sum().sort_values(ascending=False)


row_id            0
order_id          0
order_date        0
ship_date         0
ship_mode         0
customer_id       0
customer_name     0
segment           0
city              0
state             0
country           0
market            0
region            0
product_id        0
category          0
sub_category      0
product_name      0
sales             0
quantity          0
discount          0
profit            0
shipping_cost     0
order_priority    0
dtype: int64

## 4.Recommended Action ##

In [22]:
# Drop the postal_code column due to excessive missing data
df.drop(columns='Postal_code', inplace=True)


KeyError: "['Postal_code'] not found in axis"

In [26]:
# Confirm no nulls are left
df.isnull().sum().sum()


np.int64(0)

## 5.Data Type Cleaning ##

In [27]:
# Check  data types
df.dtypes


row_id             int64
order_id          object
order_date        object
ship_date         object
ship_mode         object
customer_id       object
customer_name     object
segment           object
city              object
state             object
country           object
market            object
region            object
product_id        object
category          object
sub_category      object
product_name      object
sales              int64
quantity           int64
discount           int64
profit             int64
shipping_cost      int64
order_priority    object
dtype: object

In [28]:
# Convert date columns to datetime
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)
df['ship_date'] = pd.to_datetime(df['ship_date'], dayfirst=True)

# Convert numeric fields to float
df['sales'] = df['sales'].astype(float)
df['discount'] = df['discount'].astype(float)
df['profit'] = df['profit'].astype(float)
df['shipping_cost'] = df['shipping_cost'].astype(float)


df.dtypes


row_id                     int64
order_id                  object
order_date        datetime64[ns]
ship_date         datetime64[ns]
ship_mode                 object
customer_id               object
customer_name             object
segment                   object
city                      object
state                     object
country                   object
market                    object
region                    object
product_id                object
category                  object
sub_category              object
product_name              object
sales                    float64
quantity                   int64
discount                 float64
profit                   float64
shipping_cost            float64
order_priority            object
dtype: object

## 6.String Field Cleaning (Text Standardization) ##

In [29]:
# Convert string fields to title case (e.g., Rick Hansen, New York)
text_cols = df.select_dtypes(include='object').columns

for col in text_cols:
    df[col] = df[col].astype(str).str.strip().str.title()


df[text_cols].head(3)











,order_id,ship_mode,customer_id,customer_name,segment,city,state,country,market,region,product_id,category,sub_category,product_name,order_priority
0,Ca-2012-124891,Same Day,Rh-19495,Rick Hansen,Consumer,New York City,New York,United States,Us,East,Tec-Ac-10003033,Technology,Accessories,Plantronics Cs510 - Over-The-Head Monaural Wir...,Critical
1,In-2013-77878,Second Class,Jr-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,Apac,Oceania,Fur-Ch-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",Critical
2,In-2013-71249,First Class,Cr-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,Apac,Oceania,Tec-Ph-10004664,Technology,Phones,"Nokia Smart Phone, With Caller Id",Medium


## 7: Load Cleaned Data into SQLite Database
I will now save the cleaned `orders` DataFrame into a local SQLite database (`orders_database.sqlite`) so we can later run SQL queries and create an EDR diagram.


In [30]:
import sqlite3
from sqlalchemy import create_engine


In [33]:
from sqlalchemy import create_engine

database_filename = "global_superstore.db"
engine = create_engine(f"sqlite:///{database_filename}")


In [35]:
from sqlalchemy import create_engine

database_filename = "global_superstore.db"
engine = create_engine(f"sqlite:///{database_filename}")

df.to_sql("orders", con=engine, if_exists="replace", index=False)


51290

In [36]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", con=engine)


,name
0,orders


In [37]:
import pandas as pd

query = """
SELECT category, SUM(sales) AS total_sales
FROM orders
GROUP BY category
ORDER BY total_sales DESC
"""

result = pd.read_sql(query, con=engine)
result


,category,total_sales
0,Technology,2.585116e+10
1,Furniture,3.583650e+09
2,Office Supplies,2.528468e+09
